In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.tools import tool, ToolRuntime

@tool
def read_email(runtime: ToolRuntime) -> str:
    """Read an email from the given address."""
    # take email from state
    return runtime.state["email"]

@tool
def send_email(body: str) -> str:
    """Send an email to the given address with the given subject and body."""
    # fake email sending
    return f"Email sent"

In [3]:
# To test middleware in tracing
from typing import Any
from langchain.agents import AgentState
from langgraph.runtime import Runtime
from langchain.agents.middleware import before_agent
from langchain.agents.middleware import after_agent

@before_agent
def do_nothing_before(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Do nothing before middleware."""
    messages = state["messages"]

    return {"messages": messages}

@after_agent
def do_nothing_after(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Do nothing after middleware."""
    messages = state["messages"]

    return {"messages": messages}

In [4]:
from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware

class EmailState(AgentState):
    email: str

agent = create_agent(
    model="gpt-5-nano",
    tools=[read_email, send_email],
    state_schema=EmailState,
    checkpointer=InMemorySaver(),
    middleware=[
        do_nothing_before,
        do_nothing_after,
        HumanInTheLoopMiddleware(
            interrupt_on={
                "read_email": False,
                "send_email": True,
            },
            description_prefix="Tool execution requires approval",
        ),
    ],
)

In [5]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {
        "messages": [HumanMessage(content="Please read my email and send a response immediately. Send the reply now in the same thread.")],
        "email": "Hi Seán, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, John."
    },
    config=config
)

In [6]:
from pprint import pprint

pprint(response)

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Hi '
                                                                          'John,\n'
                                                                          '\n'
                                                                          'No '
                                                                          'problem—thanks '
                                                                          'for '
                                                                          'letting '
                                                                          'me '
                                                                          'know. '
                                                                          'We '
                                                                          'can '
                                                                          'definitely '
       

In [7]:
print(response['__interrupt__'])

[Interrupt(value={'action_requests': [{'name': 'send_email', 'args': {'body': 'Hi John,\n\nNo problem—thanks for letting me know. We can definitely reschedule.\n\nWould tomorrow at 10:00 AM or 1:00 PM work for you? If neither works, please suggest a couple of times that fit your schedule and I’ll adapt.\n\nBest regards,\nSeán'}, 'description': "Tool execution requires approval\n\nTool: send_email\nArgs: {'body': 'Hi John,\\n\\nNo problem—thanks for letting me know. We can definitely reschedule.\\n\\nWould tomorrow at 10:00 AM or 1:00 PM work for you? If neither works, please suggest a couple of times that fit your schedule and I’ll adapt.\\n\\nBest regards,\\nSeán'}"}], 'review_configs': [{'action_name': 'send_email', 'allowed_decisions': ['approve', 'edit', 'reject']}]}, id='934559f6c71de2a7cc22700ac6e0029c')]


In [8]:
# Access just the 'body' argument from the tool call
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

Hi John,

No problem—thanks for letting me know. We can definitely reschedule.

Would tomorrow at 10:00 AM or 1:00 PM work for you? If neither works, please suggest a couple of times that fit your schedule and I’ll adapt.

Best regards,
Seán


## Approve

In [9]:
from langgraph.types import Command

response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}
    ), 
    config=config # Same thread ID to resume the paused conversation
)

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response immediately. Send the reply now in the same thread.', additional_kwargs={}, response_metadata={}, id='8c15ea3b-367f-475d-926d-b0b4c150fa1f'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 339, 'prompt_tokens': 167, 'total_tokens': 506, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D2o8L2u7nytnMydTeWOyFjszZaj6A', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c0219-5685-7e50-9907-e31f5ed7d8ea-0', tool_calls

## Reject

In [10]:
config = {"configurable": {"thread_id": "2"}}

response = agent.invoke(
    {
        "messages": [HumanMessage(content="Please read my email and send a response immediately. Send the reply now in the same thread.")],
        "email": "Hi Seán, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, John."
    },
    config=config
)

pprint(response)

print("-----------------------------------")
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])
print("-----------------------------------")

response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "reject",
                    # An explanation of why the request was rejected
                    "message": "No please sign off - Your merciful leader, Seán."
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Hi '
                                                                          'John,\n'
                                                                          '\n'
                                                                          'No '
                                                                          'problem—thanks '
                                                                          'for '
                                                                          'the '
                                                                          'heads '
                                                                          'up. '
                                                                          "I'm "
                                                                          'happy '
                                                                          'to '
               

## Edit

In [11]:
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "edit",
                    # Edited action with tool name and args
                    "edited_action": {
                        # Tool name to call.
                        # Will usually be the same as the original action.
                        "name": "send_email",
                        # Arguments to pass to the tool.
                        "args": {"body": "This is the last straw, you're fired!"},
                    }
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Hi '
                                                                          'John,\n'
                                                                          '\n'
                                                                          'Thanks '
                                                                          'for '
                                                                          'the '
                                                                          'heads '
                                                                          'up. '
                                                                          'I’m '
                                                                          'happy '
                                                                          'to '
                                                                          'reschedule. '
              